In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

'''
ROTH IRA - 6000
TRADITIONAL 401K - 19000
ROTH 401K - 19000
'''

'\nROTH IRA - 6000\nTRADITIONAL 401K - 19000\nROTH 401K - 19000\n'

In [29]:
import pandas as pd
import numpy as np

class Profit():
    def __init__(self, age=30, retirement_age=65, annual_contribution=19000, percent_return=1.08, percent_return_non_401k=1.08, base_salary=100000):
        self.age = age
        self.retirement_age = retirement_age
        self.annual_contribution = annual_contribution
        self.percent_return = percent_return
        self.percent_return_non_401k = percent_return_non_401k
        self.base_salary = base_salary

    def roth_401k(self, years):
        res = self.annual_contribution
        annual_list =[self.annual_contribution]
        for i in range(years):
            res = self.annual_contribution + annual_list[i] * self.percent_return
            annual_list.append(int(res))
        return annual_list

    def traditional_401k_post_tax(self, df, tax_low):
        # input dataframe with 401k_roth column
        
        res=[]
        for i in df['401k_roth']:
            res.append(int(i*tax_low))
        return res

    def tax_bracket(self):
        # if no tax bracket change, then go with roth 100% -- if bracket changes, put as much in traditional to lower tax bracket
    
        bracket_list = [9875, 40125, 85525, 163300, 207350, 518400]
        tax_list = [0.1, 0.12, 0.22, 0.24, 0.32, 0.35, 0.37]
        diff = self.base_salary - self.annual_contribution

        if diff > bracket_list[5]:
            traditional_contribution = 0
            roth_contribution = self.annual_contribution
            tax_up = tax_list[6]
            tax_low = tax_list[5]
            print('Your tax bracket is: {}%'.format(tax_up))
            print('Contribute ${} to your traditional 401k'.format(traditional_contribution))
            print('You should contribute {} to your roth 401k at your current tax rate of {}%'.format(roth_contribution, tax_up))
        else:
            for i in range(4, 0, -1):
                if bracket_list[i] < diff <= bracket_list[i+1]:
                    if self.base_salary - bracket_list[i+1] < 0:
                        tax_up = tax_list[i+1]
                        tax_low = tax_list[i]
                        traditional_contribution = 0
                        roth_contribution = self.annual_contribution
                        print('Your tax bracket is: {}%'.format(tax_up))
                        print('Contribute ${} to your traditional 401k'.format(traditional_contribution))
                        print('You should contribute {} to your roth 401k at your current tax rate of {}%'.format(roth_contribution, tax_up))
                    else:
                        tax_up = tax_list[i+2]
                        tax_low = tax_list[i+1]
                        traditional_contribution = self.base_salary - bracket_list[i+1]
                        roth_contribution = self.annual_contribution - (self.base_salary - bracket_list[i+1])
                        print('Your upper tax is: {}%'.format(tax_up))
                        print('Your lower tax is: {}%'.format(tax_low))
                        print('You can lower your tax bracket to {}% by contributing to your traditional 401k: ${}'.format(tax_low, traditional_contribution))
                        print('Contribute the rest to your roth 401k: ${}'.format(roth_contribution))
        
        trad = round(traditional_contribution / 24, 2)
        roth = round(roth_contribution / 24, 2)
        print('-------------------------------------------------------------------------------------')
        print('Bimonthly traditional 401k contribution: ${}'.format(trad))
        print('Bimonthly roth 401k contribution: ${}'.format(roth))

        return (1-tax_up), (1-tax_low)
        

    def brokerage_list(self, years, tax_up, tax_low):
        roth = self.base_salary*tax_up - self.annual_contribution
        traditional = (self.base_salary-self.annual_contribution)*tax_low
        difference = int(traditional - roth)
        
        annual_trad_list =[difference]
        for i in range(years):
            res = difference + annual_trad_list[i] * self.percent_return_non_401k
            annual_trad_list.append(int(res))
        print('Monthly brokerage contribution: ${}'.format(round(difference/12,2)))
        return annual_trad_list


    def brokerage_post_tax(self, df):
        long_term_capital_gains_tax=0.2
        res=[]
        for i in df['lost roth 401k oppurtunity_cost']:
            res.append(int(i*(1-long_term_capital_gains_tax)))
        return res


    '''
    --------------------------------------------------------------------------------------------------
    '''

    def howdy_401k(self):
        print('Your base salary is ${}'.format(self.base_salary))
        years = self.retirement_age-self.age
        
        # make the table
        df = pd.DataFrame(list(range(years+1)), columns=['years'])
        
        # find tax range
        tax_up, tax_low = self.tax_bracket()

        # create the return for 401k
        df['401k_roth'] = self.roth_401k(years)
        df['401k_traditional (pre_tax)'] = self.roth_401k(years)
        
        # create the return post tax traditional
        df['401k_traditional (post_tax)'] = self.traditional_401k_post_tax(df, tax_low)
        
        # create brokerage by getting the difference and capturing oppurtunity cost
        brokerage = self.brokerage_list(years, tax_up, tax_low)
        df['lost roth 401k oppurtunity_cost'] = brokerage    
        
        # create brokerage returns post tax
        df['lost roth 401k oppurtunity_cost (capital_gain_tax)'] = self.brokerage_post_tax(df)
        
        df['difference (trad<>roth)'] = df['401k_roth'] - (df['401k_traditional (post_tax)']+df['lost roth 401k oppurtunity_cost (capital_gain_tax)'])
        
        return df

def user_input():
    age = int(input('Current age: '))
    retirement_age = int(input('Retirment age: '))
    annual_contribution = int(input('Contribution to your 401k: '))
    percent_return = float(input('Average annual return for your 401k: '))
    percent_return_non_401k = float(input('Annual return for brokerage: '))
    base_salary = int(input('Base Salary: '))
    print('-------------------------------------------------------------------------------------')

    return age, retirement_age, annual_contribution, percent_return, percent_return_non_401k, base_salary

def run_m():
    age, retirement_age, annual_contribution, percent_return, percent_return_non_401k, base_salary = user_input()
    mod = Profit(age, retirement_age, annual_contribution, percent_return, percent_return_non_401k, base_salary)
    df = mod.howdy_401k()

    return df


# use this to run the model in your notebook or terminal with the following values as default
# mod = Profit(age=30, retirement_age=65, annual_contribution=19000, percent_return=1.075, percent_return_non_401k=1.08, base_salary=100000)
mod = Profit(base_salary=520000)
df = mod.howdy_401k()
df

Your base salary is $520000
Your upper tax is: 0.37%
Your lower tax is: 0.35%
You can lower your tax bracket to 0.35% by contributing to your traditional 401k: $1600
Contribute the rest to your roth 401k: $17400
-------------------------------------------------------------------------------------
Bimonthly traditional 401k contribution: $66.67
Bimonthly roth 401k contribution: $725.0
Monthly brokerage contribution: $1420.83


,years,401k_roth,401k_traditional (pre_tax),401k_traditional (post_tax),lost roth 401k oppurtunity_cost,lost roth 401k oppurtunity_cost (capital_gain_tax),difference (trad<>roth)
0,0,19000,19000,12350,17050,13640,-6990
1,1,39520,39520,25688,35464,28371,-14539
2,2,61681,61681,40092,55351,44280,-22691
3,3,85615,85615,55649,76829,61463,-31497
4,4,111464,111464,72451,100025,80020,-41007
5,5,139381,139381,90597,125077,100061,-51277
6,6,169531,169531,110195,152133,121706,-62370
7,7,202093,202093,131360,181353,145082,-74349
8,8,237260,237260,154219,212911,170328,-87287
9,9,275240,275240,178906,246993,197594,-101260


In [5]:
# df.to_csv('generic_35_year_plan', index=False)

57000.0

In [158]:
traditional=(100000-19000)*0.78
traditional

63180.0

In [159]:
roth = 100000*0.76 - 19000
traditional=(100000-19000)*0.78
difference = traditional - roth
difference

6180.0

In [ ]:
tax_rate = []
    
roth = base_salary*0.76 - 19000   #24% tax rate
traditional = (base_salary-19000)*0.78  #22% tax rate
difference = traditional - roth

In [96]:
df = pd.DataFrame(list(range(31)), columns=['years'])
res = 1000
annual_list =[1000]
for i in range(30):
    res = 1000 + annual_list[i] * 1.10
    annual_list.append(int(res))
df['return'] = annual_list
# df['post_tax'] = 
df

,years,return
0,0,1000
1,1,2100
2,2,3310
3,3,4641
4,4,6105
5,5,7715
6,6,9486
7,7,11434
8,8,13577
9,9,15934


In [39]:
for i in [1000] * 10:
    print(i*1.07)

1070.0
1070.0
1070.0
1070.0
1070.0
1070.0
1070.0
1070.0
1070.0
1070.0


In [50]:
annual_return = 1000
annual_list =[1000]
for i in range(9):
    annual_return += 1000 * 1.07
    annual_list.append(annual_return)

In [51]:
annual_return

10630.0

In [ ]:
def tax_bracket(base_salary):
    

In [306]:
# if no tax bracket change, then go with roth 100% -- if bracket changes, put as much in traditional to lower tax bracket
base_salary = 100000
annual_contribution = 15000

bracket_1 = 9875
bracket_2 = 40125
bracket_3 = 85525
bracket_4 = 163300
bracket_5 = 207350
bracket_6 = 518400
bracket_list = [bracket_1, bracket_2, bracket_3, bracket_4, bracket_5, bracket_6]
diff = base_salary - annual_contribution

if diff > bracket_6:
    traditional_contribution = 0
    roth_contribution = annual_contribution
else:
    for i in range(4, 0, -1):
        if bracket_list[i] < diff <= bracket_list[i+1]:
            if base_salary - bracket_list[i+1] < 0:
                traditional_contribution = 0
                roth_contribution = annual_contribution
            else:
                traditional_contribution = base_salary - bracket_list[i+1]
                roth_contribution = annual_contribution - (base_salary - bracket_list[i+1])
            
print('traditional: {}'.format(traditional_contribution))
print('roth: {}'.format(roth_contribution))

traditional: 14475
roth: 525


5
